<a href="https://colab.research.google.com/github/martaarozarena/KSchool-Master-Final-Project/blob/master/exog_varaibles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd

In [34]:
sheets=["c1_schoolclosing","c2_workplaceclosing","c3_cancelpublicevents","c4_restrictionsongatherings","c5_closepublictransport","c6_stayathomerequirements","c7_domestictravel","c8_internationaltravel","h1_publicinfocampaign","h2_testingpolicy","h3_contacttracing"]
initialdate="01-01-2020"
enddate="30-06-2020"

In [35]:
var = pd.read_excel("https://github.com/OxCGRT/covid-policy-tracker/raw/master/data/timeseries/OxCGRT_timeseries_all.xlsx",sheet_name=sheets)

In [36]:
j=1
for i in var.keys():
  var[i]=var[i].drop("CountryCode",axis=1)
  var[i].index=var[i].CountryName
  var[i]=var[i].drop("CountryName",axis=1)
  var[i]=var[i][:-3]
  var[i]=var[i].T
  var[i].index=pd.to_datetime(var[i].index,format="%d%b%Y")
  var[i]=var[i].truncate(initialdate,enddate)
  j=j+1

In [37]:
b=1
for i in sheets:
  if b==1: 
    var[i].columns="{}_".format(i[3:])+var[i].columns
    exogenas=var[i]
  else:
    concat=var[i]
    concat.columns="{}_".format(i[3:])+var[i].columns
    exogenas=pd.concat([exogenas,concat],axis=1)
  b=b+1

In [38]:
#download the mask wearing data

In [39]:
url2 = "https://drive.google.com/file/d/1WsBkpNhB5ilLsNxmOZPwdXv66m5tPDHx/view?usp=sharing"
path2 = 'https://drive.google.com/uc?export=download&id='+url2.split('/')[-2]
var_percent_mask = pd.read_csv(path2,sep=";")

In [40]:
var_percent_mask.DateTime=pd.to_datetime(var_percent_mask.DateTime)
var_percent_mask.index=var_percent_mask.DateTime
var_percent_mask.drop("DateTime",axis=1,inplace=True)

In [41]:
#add the missing dates
var_percent_mask=var_percent_mask.resample("D").mean()

In [42]:
#fill the missing dates with an interpolation
var_percent_mask.interpolate(method='linear',inplace=True)
var_percent_mask.fillna(method="bfill",inplace=True)


In [43]:
#rename the variable to be added in the general dataframe and choose the dates
var_percent_mask.columns="mask_"+var_percent_mask.columns
var_percent_mask=var_percent_mask.truncate(initialdate,enddate)

In [44]:
#added to the generales dataframe and the missing initial values add 0s cause people was not wearing masks at the beggining
exogenas=pd.concat([exogenas,var_percent_mask],axis=1)
exogenas.fillna(0,inplace=True)

In [45]:
#flights per day
url3 = "https://drive.google.com/file/d/16AKDzYIdaPH2omW2R30r6Gx0sKNU8-dc/view?usp=sharing"
path3 = 'https://drive.google.com/uc?export=download&id='+url3.split('/')[-2]
flights = pd.read_excel(path3,sheet_name="DATA",date_parser="FLT_DATE",index_col ="FLT_DATE")

In [46]:
flights["STATE_NAME"].unique()

array(['Belgium', 'Germany', 'Estonia', 'Finland', 'United Kingdom',
       'Netherlands', 'Ireland', 'Denmark', 'Luxembourg', 'Norway',
       'Poland', 'Sweden', 'Latvia', 'Lithuania', 'Spain', 'Albania',
       'Bulgaria', 'Cyprus', 'Croatia', 'France', 'Greece', 'Hungary',
       'Italy', 'Slovenia', 'Czech Republic', 'Malta', 'Austria',
       'Portugal', 'Bosnia and Herzegovina', 'Romania', 'Switzerland',
       'Turkey', 'Moldova', 'The former Yugoslav Republic of Macedonia',
       'Serbia', 'Montenegro', 'Slovakia', 'Armenia', 'Georgia',
       'Ukraine', 'Morocco', 'Israel', 'Republic of North Macedonia',
       'North Macedonia'], dtype=object)

In [47]:
for i in flights["STATE_NAME"].unique():
  flights_country=flights[flights["STATE_NAME"]==i]
  flights_country=flights_country.resample("1D").sum()
  flights_country=flights_country['FLT_ARR_1'].truncate(initialdate,enddate)
  flights_country.rename("Flights_{}".format(i),inplace=True)
  exogenas=pd.concat([exogenas,flights_country],axis=1)

In [48]:
exogenas

,schoolclosing_Aruba,schoolclosing_Afghanistan,schoolclosing_Angola,schoolclosing_Anguilla,schoolclosing_Albania,schoolclosing_Andorra,schoolclosing_United Arab Emirates,schoolclosing_Argentina,schoolclosing_Australia,schoolclosing_Austria,schoolclosing_Azerbaijan,schoolclosing_Burundi,schoolclosing_Belgium,schoolclosing_Benin,schoolclosing_Burkina Faso,schoolclosing_Bangladesh,schoolclosing_Bulgaria,schoolclosing_Bahrain,schoolclosing_Bosnia and Herzegovina,schoolclosing_Belarus,schoolclosing_Belize,schoolclosing_Bermuda,schoolclosing_Bolivia,schoolclosing_Brazil,schoolclosing_Barbados,schoolclosing_Brunei,schoolclosing_Bhutan,schoolclosing_Botswana,schoolclosing_Central African Republic,schoolclosing_Canada,schoolclosing_Switzerland,schoolclosing_Chile,schoolclosing_China,schoolclosing_Cote d'Ivoire,schoolclosing_Cameroon,schoolclosing_Democratic Republic of Congo,schoolclosing_Congo,schoolclosing_Colombia,schoolclosing_Cape Verde,schoolclosing_Costa Rica,...,Flights_United Kingdom,Flights_Netherlands,Flights_Ireland,Flights_Denmark,Flights_Luxembourg,Flights_Norway,Flights_Poland,Flights_Sweden,Flights_Latvia,Flights_Lithuania,Flights_Spain,Flights_Albania,Flights_Bulgaria,Flights_Cyprus,Flights_Croatia,Flights_France,Flights_Greece,Flights_Hungary,Flights_Italy,Flights_Slovenia,Flights_Czech Republic,Flights_Malta,Flights_Austria,Flights_Portugal,Flights_Bosnia and Herzegovina,Flights_Romania,Flights_Switzerland,Flights_Turkey,Flights_Moldova,Flights_The former Yugoslav Republic of Macedonia,Flights_Serbia,Flights_Montenegro,Flights_Slovakia,Flights_Armenia,Flights_Georgia,Flights_Ukraine,Flights_Morocco,Flights_Israel,Flights_Republic of North Macedonia,Flights_North Macedonia
2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2034,628,308,223,63,683,454,298,102,63,2027,35,61,74,37,1700,308,140,1299,14,154,53,317,479,10,96,490,1101,19,NaN,63,9,14,37,29,105,283,195,21.0,NaN
2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2650,683,394,319,82,1045,629,385,123,94,2535,41,95,74,55,2188,383,168,1587,26,224,77,480,554,12,144,613,1242,31,NaN,94,14,21,37,40,129,303,256,29.0,NaN
2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2777,713,404,309,92,932,597,380,125,78,2556,40,88,69,56,2289,415,179,1644,29,210,72,493,594,18,177,644,1273,34,NaN,75,19,26,35,43,142,309,177,24.0,NaN
2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2354,607,339,231,76,453,489,253,94,74,2561,36,86,89,43,2070,356,131,1434,20,155,75,502,591,11,135,622,1253,34,NaN,76,12,16,42,38,129,344,169,24.0,NaN
2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2705,709,388,298,98,845,565,370,102,72,2475,45,82,81,48,2283,381,164,1576,23,213,76,463,577,14,167,674,1285,34,NaN,87,15,19,43,44,145,326,254,24.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-26,0.0,3.0,3.0,2.0,2.0,0.0,3.0,3.0,2.0,0.0,3.0,0.0,2.0,2.0,0.0,3.0,2.0,3.0,3.0,0.0,3.0,3.0,3.0,3.0,2.0,2.0,3.0,2.0,3.0,3.0,0.0,3.0,3.0,0.0,2.0,3.0,2.0,3.0,3.0,3.0,...,482,214,91,78,48,300,134,89,32,32,688,20,46,33,31,998,244,56,391,30,59,8,129,116,7,63,187,474,11,NaN,51,3,16,6,9,32,36,29,NaN,3.0
2020-06-27,0.0,3.0,3.0,2.0,2

In [49]:
#creating the csv file
exogenas.to_csv('exogenas.csv')